In [1]:
import rainbowhat as rh
from enum import Enum
import subprocess
import re
import time
import itertools

In [2]:
class RGBColors(Enum):
   RED = (50, 0, 0)
   YELLOW = (50, 50, 0)
   PINK = (50, 10, 12)
   GREEN = (0, 50, 0)
   PURPLE = (50, 0, 50)
   ORANGE = (50, 22, 0)
   BLUE = (0, 0, 50)

In [3]:
def run_rainbow(it):
    for pixel, color in it:
        rh.rainbow.clear()
        rh.rainbow.set_pixel(pixel, *(color.value))
        rh.rainbow.show()
        time.sleep(0.1)


In [4]:
def open_rainbow():
    colours = list(RGBColors)
    for pixel in itertools.chain(reversed(range(4)),range(4)):
        rh.rainbow.clear()
        rh.rainbow.set_pixel(pixel, *(colours[pixel].value))
        rh.rainbow.set_pixel(-pixel+6, *(colours[-pixel+6].value))
        rh.rainbow.show()
        time.sleep(0.4)
    
    rh.rainbow.clear()
    rh.rainbow.show()


In [5]:
def right_to_left():
    run_rainbow(zip(range(7), RGBColors))

In [6]:
def left_to_right():
    run_rainbow(reversed(tuple(zip(range(7), RGBColors))))

In [7]:

rh.rainbow.clear()
rh.display.clear()

for idx in range(1, 11):
    rh.display.print_str(f'{idx}')
    rh.display.show()

    if (idx % 2) == 1:
        left_to_right()
    else:
        right_to_left()
    

rh.rainbow.clear()
rh.rainbow.show()
rh.display.print_str('DONE')
rh.display.show()

In [8]:
TEMP_EXTRACTOR = re.compile(r"[^=]+=(\d+(?:\.\d+))'C\n$")

In [9]:
def get_temp(real_temp):
    cpu_temp_str = subprocess.check_output("vcgencmd measure_temp", shell=True).decode("utf-8")
    temp = rh.weather.temperature()
    cpu_temp = float(TEMP_EXTRACTOR.match(cpu_temp_str)[1])
#     print(temp, cpu_temp)
    FACTOR = -(cpu_temp - temp)/(real_temp - temp)
    print('FACTOR', FACTOR)
    FACTOR = 0.6976542896954581
    
    return temp - ((cpu_temp - temp)/FACTOR)

In [10]:
@rh.touch.A.press()
def touch_a(channel):
    rh.lights.rgb(1, 0, 0)
    rh.display.clear()
    rh.display.print_str("TEMP")
    rh.display.show()
    time.sleep(1)
    rh.display.print_float(get_temp(20.5))
    rh.display.show()
    open_rainbow()
    
def release(channel):
    rh.display.clear()
    rh.display.show()
    rh.rainbow.clear()
    rh.rainbow.show()
    rh.lights.rgb(0, 0, 0)

In [11]:
@rh.touch.B.press()
def touch_b(channel):
    rh.lights.rgb(0, 1, 0)
    rh.display.clear()
    rh.display.print_str(" B  ")
    rh.display.show()
    time.sleep(0.5)
    right_to_left()


In [12]:
@rh.touch.C.press()
def touch_c(channel):
    rh.lights.rgb(0, 0, 1)
    rh.display.clear()
    rh.display.print_str("   C")
    rh.display.show()
    time.sleep(0.5)
    left_to_right()


In [13]:
rh.touch.A.release()(release)
rh.touch.B.release()(release)
rh.touch.C.release()(release)